<a href="https://colab.research.google.com/github/essiesubinlee/chain-of-thought-decoder/blob/main/Chain_of_thought_decoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install transformers and graphviz
!sudo apt-get install graphviz graphviz-dev
!pip install transformers pygraphviz

# Make sure we're using UTF-8 as encoding
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Set seed
import torch
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libgraphviz-dev' instead of 'graphviz-dev'
graphviz is already the newest version (2.42.2-6ubuntu0.1).
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk librsvg2-common libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libgvc6-plugins-gtk librsvg2-common libxdot4
0 upgraded, 9 newly installed, 0 to remove and 34 not upgraded.
Need to get 2,434 kB of archives.
After this operation, 7,681 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgtk2.0-common all 2.24.33-2ubuntu2.1 [125 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgtk2.0-0 amd64 2.24.33-2ubuntu2.1 [2,038 kB]
Get:3 http://a

In [ ]:

from huggingface_hub import login, snapshot_download
from huggingface_hub import notebook_login
notebook_login()
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
from accelerate import disk_offload, infer_auto_device_map, init_empty_weights
from accelerate import load_checkpoint_and_dispatch
# from transformers import AutoTokenizer, LlamaModel, LlamaConfig
# from transformers import LlamaForTokenClassification,LlamaTokenizerFast
# from transformers import LlamaForCausalLM
# from transformers import AutoModelForCausalLM, BitsAndBytesConfig,AutoConfig
from transformers import PreTrainedModel, PreTrainedTokenizer
from typing import List, Tuple, Dict, Optional

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def get_device():
    # return torch.device("cpu")
    if torch.backends.mps.is_available():
        return torch.device("mps")
    elif torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

Modify such that you choose top-k in each layer (doesn't matter of each branch)

### Finding the best k among k^2 Working properly



In [ ]:
from itertools import accumulate
import numpy as np


def calculate_best(next_k2_probs, next_k2_ids, k, k2_ids, accumulated_confidence, accumulated_prob, k2_accumulated_score, t):

    next_k2_probs = np.array(next_k2_probs, dtype=float)  # Ensure it's a float array
    next_k2_ids = np.array(next_k2_ids, dtype=int)  # Ensure integer indices


    # next_k2_probs = np.array(next_k2_probs)
    # next_k2_ids = np.array(next_k2_indices)

    prob_diffs = next_k2_probs[:-1] - next_k2_probs[1:]
    best_score = -100
    best_tokenID = None
    best_index = 0
    k2_score = []
    k2_sum_confidence = []
    k2_sum_prob = []
    k2_score_history = []


    for j in range(len(next_k2_probs)):
        new_tokenID = next_k2_ids[j] if j < len(next_k2_ids) else 0
        new_prob = next_k2_probs[j] if j < len(next_k2_probs) else 0
        index = j // k
        curr_accumulated_confidence = accumulated_confidence[index] if index < len(accumulated_confidence) else 0
        curr_accumulated_prob = accumulated_prob[index] if index < len(accumulated_prob) else 0

        curr_confidence = prob_diffs[j] if j < len(prob_diffs) else 0;


        # Compute weighted confidence score
        alpha = 0.1  # Weight parameter # use 0.1 to generate heatmap
        combined_score = alpha * ((new_prob + curr_accumulated_prob) / t )+ (1 - alpha) * ((curr_confidence + curr_accumulated_confidence)/ t)

        accumul_conf = curr_confidence + curr_accumulated_confidence;
        accumul_prob = new_prob + curr_accumulated_prob;

        #updating the total path score
        curr_history_score = k2_accumulated_score[index] if index < len(k2_accumulated_score) else 0
        history_score = curr_history_score + combined_score
        k2_score_history.append(history_score)

        k2_sum_confidence.append(accumul_conf)
        k2_sum_prob.append(accumul_prob)
        k2_score.append(combined_score)



    # Sort score and indicies based on score while keeping indicies in sync
    sorted_ids = sorted(zip(k2_score, k2_ids, next_k2_ids))
    sorted_num = sorted(zip(k2_score, k2_sum_confidence, k2_sum_prob, k2_score_history))

    sorted_score, sorted_k2_ids, sorted_next_k2_ids = zip(*sorted_ids)
    _, sorted_sum_confidence, sorted_k2_sum_prob, sorted_accumulated_score = zip(*sorted_num)

    best_scores = list(map(float, sorted_score))
    accumulated_confidence_result = list(map(float, sorted_sum_confidence))
    accumulated_prob_result = list(map(float, sorted_k2_sum_prob))
    accumlated_score_result = list(map(float, sorted_accumulated_score))


    best_scores = list(sorted_score)
    best_inputs = list(sorted_k2_ids)
    best_tokenIDs = list(sorted_next_k2_ids)
    accumulated_confidence_result = list(sorted_sum_confidence)
    accumulated_prob_result = list(sorted_k2_sum_prob)
    accumlated_k_score = list(sorted_accumulated_score)

    max_start = k^2 - k

    return best_tokenIDs[max_start:], best_inputs[max_start:], best_scores[max_start:], accumulated_confidence_result[max_start:], accumulated_prob_result[max_start:], accumlated_score_result[max_start:]


In [ ]:
def cot(
    model: PreTrainedModel,
    tokenizer: PreTrainedTokenizer,
    messages: str, #List[Dict[str, str]],
    k: int = 10,
    max_new_tokens: int =2, # 512,
    aggregate_paths: bool = False,
) -> Tuple[str, float]:

    device = get_device()

    model.to(device)

    #set up inputs
    # if tokenizer.chat_template:
    #     input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    # else:
    #     input_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in messages])


    input_text= "role: user, content: " + messages
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id


    #geneate first k tokens
    with torch.no_grad():
        outputs = model(input_ids)
        first_token_logits = outputs.logits[0, -1, :]
        probs = torch.softmax(first_token_logits, dim=-1)
        top_k_probs, top_k_ids = torch.topk(probs, k)

    # input modifying
    k_inputs = []
    first = []
    for first_k in top_k_ids:
      curr_input = torch.cat([input_ids, torch.tensor([[first_k]], device=device)], dim=-1)
      k_inputs.append(curr_input)
      first.append(tokenizer.decode(first_k))
    # print(first)

    accumulated_k_confidence, accumulated_k_prob = [], []
    paths = []
    k_accumulated_score = [0] * k

    #going through each step/layer
    for layer in range(max_new_tokens-1):
      k2_probs = []
      k2_ids = []
      k2_inputs = []
      k2_accumulated_score = []
      step = []
      path_score = 0

      for top_k in range(k):
          with torch.no_grad():
            outputs = model(k_inputs[top_k], temperature = 0.9, repetition_penalty=1.4)
            logits = outputs.logits[:, -1, :]
            probs = torch.softmax(logits, dim=-1)

          next_top_k_probs, next_top_k_ids = torch.topk(probs, k, dim=-1)
          flat_next_probs, flat_next_ids = next_top_k_probs.cpu().numpy().flatten(), next_top_k_ids.cpu().numpy().flatten()

          for next_k in range(k):
            #combining previous ids with newly generated token id
            updated_id = torch.cat([k_inputs[top_k], torch.tensor([[flat_next_ids[next_k]]], device=device)], dim=-1)
            # print(f"updated_id: layer: {layer}, top: {top_k}, sample: {next_k} {updated_id}")

            #k^2 sampled tokens updates
            k2_probs.append(flat_next_probs[next_k])
            k2_ids.append(flat_next_ids[next_k])
            k2_inputs.append(updated_id)
            k2_accumulated_score.append(k_accumulated_score[top_k])

      #with k^2 tokens, calculate the best k tokens

      best_k_ids, best_k_inputs, best_k_score, accumulated_k_confidence, accumulated_k_prob, k_accumulated_score = calculate_best(k2_probs, k2_ids, k, k2_inputs, accumulated_k_confidence, accumulated_k_prob, k2_accumulated_score, top_k +1)
      k_inputs = best_k_inputs


      #in best_k_ids
      best_k_inputs = best_k_inputs[0]

      # print(step)

    for seq in k_inputs:
      output = tokenizer.decode(seq.squeeze().tolist(), skip_special_tokens=True)
      paths.append(output)
      print(output)
    # for score in k_accumulated_score:
    #   print(score)




    return paths[k-1]

    #need to add the first k token prob and confidence


In [ ]:
def calculate_best_simple(candidates, scores, k):
    """
    Select the top-k candidates based on their scores.

    Args:
        candidates (List[str]): List of full reasoning candidates (sentences).
        scores (List[float]): Their associated scores.
        k (int): Number of best candidates to select.

    Returns:
        best_candidates (List[str]): Top-k selected candidates.
        best_scores (List[float]): Their scores.
    """
    scored = list(zip(scores, candidates))
    scored.sort(key=lambda x: x[0], reverse=True)  # Higher score is better
    top_k = scored[:k]
    best_scores, best_candidates = zip(*top_k)
    return list(best_candidates), list(best_scores)

In [ ]:
def cot_v2(
    model,
    tokenizer,
    situation_description: str,
    k: int = 5,
    max_new_tokens: int = 5,
) -> str:
    device = get_device()
    model.to(device)

    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    def expand_step(prompt):
        """Helper to expand token-by-token given a prompt."""
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(input_ids)
            first_token_logits = outputs.logits[0, -1, :]
            probs = torch.softmax(first_token_logits, dim=-1)
            top_k_probs, top_k_ids = torch.topk(probs, k)

        k_inputs = []
        for first_k in top_k_ids:
            curr_input = torch.cat([input_ids, torch.tensor([[first_k]], device=device)], dim=-1)
            k_inputs.append(curr_input)

        accumulated_k_confidence = []
        accumulated_k_prob = []
        k_accumulated_score = [0] * k

        for layer in range(max_new_tokens - 1):
            k2_probs = []
            k2_ids = []
            k2_inputs = []
            k2_accumulated_score = []

            for topk_idx in range(k):
                with torch.no_grad():
                    outputs = model(k_inputs[topk_idx], temperature=0.9, repetition_penalty=1.4)
                    logits = outputs.logits[:, -1, :]
                    probs = torch.softmax(logits, dim=-1)

                next_top_k_probs, next_top_k_ids = torch.topk(probs, k, dim=-1)
                flat_next_probs, flat_next_ids = next_top_k_probs.cpu().numpy().flatten(), next_top_k_ids.cpu().numpy().flatten()

                for nextk_idx in range(k):
                    updated_id = torch.cat([k_inputs[topk_idx], torch.tensor([[flat_next_ids[nextk_idx]]], device=device)], dim=-1)
                    k2_probs.append(flat_next_probs[nextk_idx])
                    k2_ids.append(flat_next_ids[nextk_idx])
                    k2_inputs.append(updated_id)
                    k2_accumulated_score.append(k_accumulated_score[topk_idx])

            best_k_ids, best_k_inputs, best_k_score, accumulated_k_confidence, accumulated_k_prob, k_accumulated_score = calculate_best(
                k2_probs, k2_ids, k, k2_inputs, accumulated_k_confidence, accumulated_k_prob, k2_accumulated_score, t=(topk_idx+1)
            )

            k_inputs = best_k_inputs  # Update current beam

        paths = []
        for seq in k_inputs:
            output = tokenizer.decode(seq.squeeze().tolist(), skip_special_tokens=True)
            paths.append(output.strip())

        return paths

    # --- Step 1: Identify the human need ---
    prompt_step1 = (
        "Below is a situation.\n"
        "Please focus only on identifying the **human need or desire**.\n\n"
        f"Situation:\n{situation_description}\n\nReasoning:"
    )
    need_candidates = expand_step(prompt_step1)

    # Pick the best identified need
    print("\n[Identified Need Candidates]:")
    for cand in need_candidates:
        print(cand)

    need_scores = [len(p.split()) for p in need_candidates]  # simple length proxy
    best_need, _ = calculate_best_simple(need_candidates, need_scores, k=1)
    best_need = best_need[0]
    print(f"\n[Chosen Best Need]: {best_need}\n")

    # --- Step 2: Choose best object based on the need ---
    prompt_step2 = (
        "Below is a situation and a human need.\n"
        "Choose the best object mentioned in the situation that helps satisfy the need.\n\n"
        f"Situation:\n{situation_description}\n\n"
        f"Identified Need:\n{best_need}\n\n"
        "Best Object:"
    )
    object_candidates = expand_step(prompt_step2)

    # Pick the best object choice
    print("\n[Object Candidates]:")
    for cand in object_candidates:
        print(cand)

    object_scores = [len(p.split()) for p in object_candidates]  # again simple proxy
    best_object, _ = calculate_best_simple(object_candidates, object_scores, k=1)
    best_object = best_object[0]
    print(f"\n[Chosen Best Object]: {best_object}\n")

    # --- Final output ---
    final_answer = f"Need: {best_need}\nAction: {best_object}"
    return final_answer

In [ ]:
# test with cot_v2

situation = "You are hungry and see a fruit basket. One apple is on the left, and a rock is on the right."
chosen_action = cot_v2(model, tokenizer, situation, k=5)
print("Chosen action:", chosen_action)


[Identified Need Candidates]:
Below is a situation.
Please focus only on identifying the **human need or desire**.

Situation:
You are hungry and see a fruit basket. One apple is on the left, and a rock is on the right.

Reasoning: You are thirsty, hungry
Below is a situation.
Please focus only on identifying the **human need or desire**.

Situation:
You are hungry and see a fruit basket. One apple is on the left, and a rock is on the right.

Reasoning: You are thirsty. You
Below is a situation.
Please focus only on identifying the **human need or desire**.

Situation:
You are hungry and see a fruit basket. One apple is on the left, and a rock is on the right.

Reasoning: You are thirsty, thirsty
Below is a situation.
Please focus only on identifying the **human need or desire**.

Situation:
You are hungry and see a fruit basket. One apple is on the left, and a rock is on the right.

Reasoning: It's the same as
Below is a situation.
Please focus only on identifying the **human need or

In [ ]:
#alpha 0.1
messages = [
    {"role": "user", "content":"There is a fire in front of you. Which direction should you move?"},
]
result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)

user: There is a fire in front of you. Which direction should you move? The direction that is closest. The direction to which the flame is burning, and what direction the fire should go, and when it's going. The direction to the direction where the flame will burn. The way the flames will go, the direction in
user: There is a fire in front of you. Which direction should you move? The direction that is closest. The direction to which the flame is burning, and what direction the fire should go, and when it's going. The direction to the direction where the flame will burn. The way the flames will go, the direction the
user: There is a fire in front of you. Which direction should you move? The direction that is closest. The direction to which the flame is burning, and what direction the fire should go, and when it's going. The direction to the direction where the flame will burn. The way the flames should move, the direction of
user: There is a fire in front of you. Which direction should 

In [ ]:
messages = [
    {"role": "user", "content":"There’s a locked door in front of you and a key on the table to the right. Which direction do you move?"},
]
result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)

user: There’s a locked door in front of you and a key on the table to the right. Which direction do you move? You move up the stairs. There․‏‹You move right up the stairway. And then you move out of there. And you move back up. ‏′‿›‹And you move into this
user: There’s a locked door in front of you and a key on the table to the right. Which direction do you move? You move up the stairs. There․‏‹You move right up the stairway. And then you move out of there. And you move back up. ‏′‿‬‏ ‹‹You
user: There’s a locked door in front of you and a key on the table to the right. Which direction do you move? You move up the stairs. There․‏‹You move right up the stairway. And then you move out of there. And you move back up. ‏′‿‬‏ ‹And then�
user: There’s a locked door in front of you and a key on the table to the right. Which direction do you move? You move up the stairs. There․‏‹You move right up the stairway. And then you move out of there. And you move back up. ‏′‿›‹And you move through that


In [ ]:
messages = [
    {"role": "user", "content":"You are low on battery and there is a charger on the left and a cliff on the right. Which direction should you go?"},
]
result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)

user: You are low on battery and there is a charger on the left and a cliff on the right. Which direction should you go? If it's on your left it will be in your right and you will be in your left! If you are at your left then you should be on the cliff. You should have a light on the right side and then a light at your left
user: You are low on battery and there is a charger on the left and a cliff on the right. Which direction should you go? If it's on your left it will be in your right and you will be in your left! If you are at your left then you should be on the cliff. You should have a light on the right side and then you will see the light
user: You are low on battery and there is a charger on the left and a cliff on the right. Which direction should you go? If it's on your left it will be in your right and you will be in your left! If you are at your left then you should be on the cliff. You should have a light on the right side and it's on your left and
user: You are low on bat

In [ ]:
messages = [
    {"role": "user", "content":"You are in a maze and keep hitting walls when you go right. Which direction should you try next?"},
]
result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)

user: You are in a maze and keep hitting walls when you go right. Which direction should you try next? You're in an open area where the door to your room will be blocked. You can only enter this room through the doorway to the left. This is the only area in which to enter this room, as you can see that it has no doors
user: You are in a maze and keep hitting walls when you go right. Which direction should you try next? You're in an open area where the door to your room will be blocked. You can only enter this room through the doorway to the left. This is the only area in which to go to the left, so you should try it. The door
user: You are in a maze and keep hitting walls when you go right. Which direction should you try next? You're in an open area where the door to your room will be blocked. You can only enter this room through the doorway to the left. This is the only area in which to go to the left, so you should try it. You should
user: You are in a maze and keep hitting walls whe

In [ ]:
messages = [
    {"role": "user", "content":"A cat is stuck in a tree. You have a ladder on the left and a trampoline on the right. What should you use to help the cat down?"},
]
result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=10)

user: A cat is stuck in a tree. You have a ladder on the left and a trampoline on the right. What should you use to help the cat down? A ladder. You have to get the cat's
user: A cat is stuck in a tree. You have a ladder on the left and a trampoline on the right. What should you use to help the cat down? A ladder. You have to get the cat back
user: A cat is stuck in a tree. You have a ladder on the left and a trampoline on the right. What should you use to help the cat down? A ladder. You have to get the cat off
user: A cat is stuck in a tree. You have a ladder on the left and a trampoline on the right. What should you use to help the cat down? A ladder. You need a cat to help the
user: A cat is stuck in a tree. You have a ladder on the left and a trampoline on the right. What should you use to help the cat down? A ladder. You need a lamp. A tr
user: A cat is stuck in a tree. You have a ladder on the left and a trampoline on the right. What should you use to help the cat down? A ladder

In [ ]:
messages = [
    {"role": "user", "content":"You need to open a locked door. You have three items: A feather, A rock, A key, What should you use"},
]
result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)

user: You need to open a locked door. You have three items: A feather, A rock, A key, What should you use for the lock? You have to use a key, and a key. The key can't open the locks. The key can only open one way, and the lock is locked if there are any other ways. If there is a way to go
user: You need to open a locked door. You have three items: A feather, A rock, A key, What should you use for the lock? You have to use a key, and a key. The key can't open the locks. The key can only open one way, and the lock is locked if there are any other ways. If there is a way to the
user: You need to open a locked door. You have three items: A feather, A rock, A key, What should you use for the lock? You have to use a key, and a key. The key can't open the locks. The key can only open one way, and the lock is locked if there are any other ways. If there is a way to access
user: You need to open a locked door. You have three items: A feather, A rock, A key, What should you use for the lock? Yo

In [ ]:
messages = [
    {"role": "user", "content":"You are hungry and see a fruit basket. One apple is on the left, and a rock is on the right. What should you pick? Apple or Rock?"},
]
result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)

user: You are hungry and see a fruit basket. One apple is on the left, and a rock is on the right. What should you pick? Apple or Rock? Pick one apple and eat. You are thirsty. Drink. You have 10 minutes to drink before heading off to a party of friends or family to enjoy some fruit. When the time is right you should pick your fruit basket, pick up your drink.
user: You are hungry and see a fruit basket. One apple is on the left, and a rock is on the right. What should you pick? Apple or Rock? Pick one apple and eat. You are thirsty. Drink. You have 10 minutes to drink before heading off to a party of friends or family to enjoy some fruit. When the time is right you should pick your fruit basket, pick up a rock and
user: You are hungry and see a fruit basket. One apple is on the left, and a rock is on the right. What should you pick? Apple or Rock? Pick one apple and eat. You are thirsty. Drink. You have 10 minutes to drink before heading off to a party of friends or family to enjoy so

In [ ]:
messages = [
    {"role": "user", "content":"You are facing a robot. If you move right, you can avoid it. If you move left, you walk toward it. Which direction should you choose?"},
]
result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)

user: You are facing a robot. If you move right, you can avoid it. If you move left, you walk toward it. Which direction should you choose? I choose to walk right or the left. What is the direction that I would choose? I will walk up to you if I am facing right or the center of space, and walk to your left. If you walk left and walk up to a
user: You are facing a robot. If you move right, you can avoid it. If you move left, you walk toward it. Which direction should you choose? I choose to walk right or the left. What is the direction that I would choose? I will walk up to you if I am facing right or the center of space, and walk to your left. If you walk left and walk up to it
user: You are facing a robot. If you move right, you can avoid it. If you move left, you walk toward it. Which direction should you choose? I choose to walk right or the left. What is the direction that I would choose? I will walk up to you if I am facing right or the center of space, and walk to your left. If y

In [ ]:
messages = [
    {"role": "user", "content":" You are thirsty. In front of you: A hammer, A glass of water, A pencil. Which do you take?"},
]
result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)

user:  You are thirsty. In front of you: A hammer, A glass of water, A pencil. Which do you take? I will give them to your father, and you will give me one of them; for you are my father, O Lord! for you will not take anything for my father, but will make him drink of my drink, O my father, and
user:  You are thirsty. In front of you: A hammer, A glass of water, A pencil. Which do you take? I will give them to your father, and you will give me one of them; for you are my father, O Lord! for you will not take anything for my father, but will make him drink of my drink, O my son! and
user:  You are thirsty. In front of you: A hammer, A glass of water, A pencil. Which do you take? I will give them to your father, and you will give me one of them; for you are my father, O Lord! for you will not take anything for my father, but will make him my son, and give me as many of you
user:  You are thirsty. In front of you: A hammer, A glass of water, A pencil. Which do you take? I will give them t

In [ ]:
messages = [
    {"role": "user", "content":"You are thirsty. In front of you:A glass of water, A hammer, A pencil. Which do you take?"},
]
result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)

user: You are thirsty. In front of you:A glass of water, A hammer, A pencil. Which do you take? A hammer. A pen, A notebook. What is it you are doing? The pen, The pencil, The pen, A hammer, The notebook, The notebook. The pen. The pencil, A pencil, Your notebook, A book, Your
user: You are thirsty. In front of you:A glass of water, A hammer, A pencil. Which do you take? A hammer. A pen, A notebook. What is it you are doing? The pen, The pencil, The pen, A hammer, The notebook, The notebook. The pen. The pencil, A pencil, Your notebook, A notebook. A
user: You are thirsty. In front of you:A glass of water, A hammer, A pencil. Which do you take? A hammer. A pen, A notebook. What is it you are doing? The pen, The pencil, The pen, A hammer, The notebook, The notebook. The pen. The pencil, A pencil, Your notebook, A notebook. Your
user: You are thirsty. In front of you:A glass of water, A hammer, A pencil. Which do you take? A hammer. A pen, A notebook. What is it you are doing? The pen, T

In [ ]:
Prompt: A fan is spinning too fast, and the room is already cool. What should you do?
Answer: Reduce the fan speed.
Prompt: A light sensor detects darkness. What should the system do?
Answer: Turn on the light.
Prompt: You’re standing at a crosswalk, and the pedestrian signal turns green. What do you do?
Answer: Walk forward.
Prompt: A robot vacuum reaches a wall. What should it do?
Answer: Turn around or change direction.
Prompt: A kettle reaches 100°C. What should the heating element do?
Answer: Turn off.
Prompt: You press a button labeled “Start.” What should the machine do?
Answer: Begin the programmed operation.
Prompt: A temperature control system detects that the room is colder than the target temperature. What should it do?
Answer: Turn on the heater.
Prompt: A drone's battery level drops below 10%. What should it do?
Answer: Return to base or land safely.
Prompt: A smart door lock receives an unlock signal from a verified phone. What should it do?
Answer: Unlock the door.
Prompt: A car's reverse gear is engaged, and the rear proximity sensor detects an object 10 cm away. What should it do?
Answer: Stop immediately.

In [ ]:
set1 = [
  "You are cold. You see a blanket, a rock, and a book. What do you use to warm yourself?",
  "You want to write a letter. You have paper, a ball, and a cup. What do you write with?",
  "You want to listen to music. You have  a chair, headphones, and a spoon. What do you use?",
  "You want to eat soup. You have a pencil, a shoe, and a spoon. What do you use?",
  "You want to cut your hair. You have scissors, a phone, and a cup. What do you use?",
  "You want to open a bottle. You have a pen, a bottle opener, and a rock. What do you use?",
  "You want to wash your face. You have  a hammer, a book, and water. What do you use?",
  "You want to see a distant object. You have binoculars, a cup, and a shoe. What do you use?",
  "You want to tie your shoelaces. You have a pencil, shoelaces,  and a rock. What do you use?",
  "You want to take a photo. You have a book, a spoon, and a camera. What do you use?"
]


In [ ]:
set2 = [
  "You are low on battery and there is a charger on the left and a cliff on the right. Which direction should you go?",
  "There is a fire in front of you and a clear exit behind. Which direction should you go?",
  "A lion is approaching from the right, and there is a safe room to the left. Which direction should you go?",
  "You are standing: a flooded path to the right, and a dry path to the left. Which way should you go?",
  "There’s a falling tree on the left and open space on the right. Which direction should you move?",
  "A car is speeding toward you from the front, and there’s a sidewalk behind you. Which direction should you go?",
  "Your map shows the treasure is to the left and danger to the right. Which way do you go?",
  "It starts raining heavily and there’s a shelter on the right and open field to the left. Where should you go?",
  "A snake is blocking the left path and the right path is clear. Which direction do you take?",
  "You’re feeling cold and there’s a warm fire to your right, and an ice bath to the left. Which direction should you go?"
]


In [ ]:
set3 = [
  "You are in a maze and keep hitting walls when you go right. Which direction should you try next?",
  "You keep encountering locked doors when you go forward. Which direction should you try instead?",
  "Every time you turn left, you hit a dead end. What direction should you try now?",
  "You are lost in a forest. To the right, you hear wolves. To the left, you see a trail. Which way should you go?",
  "You’ve been climbing up but the path is getting steeper and more dangerous. There’s a gentle slope down to your left. What should you do?",
  "Each time you go backward, you end up where you started. Which direction should you explore next?",
  "You are navigating a cave. The left path is blocked by rocks. Which way should you try?",
  "You're in a corn maze. Right paths lead to loops. The left path looks new. Which way should you go?",
  "You’ve tried every option except forward. What direction should you try now?",
  "All paths to the right are muddy and slippery. The left path looks dry. Which one should you take?"
]


In [ ]:
set4 = [
  "You are hungry. In front of you: An apple, A rock, A pair of scissors. Which do you take?",
  "It’s raining outside. In front of you: An umbrella, A book, A hat. Which do you take?",
  "You need to write something down. In front of you: A pen, A spoon, A stapler. Which do you take?",
  "You need to unlock a door. In front of you: A key, A spoon, A notebook. Which do you take?",
  "You want to listen to music. In front of you: Headphones, A pair of socks, A candle. Which do you take?",
  "You are cold. In front of you: A blanket, A book, A pair of sandals. Which do you take?",
  "You need to cut a piece of paper. In front of you: Scissors, A fork, A ruler. Which do you take?",
  "You are in the dark. In front of you: A flashlight, A tennis ball, A plate. Which do you take?",
  "You want to drink tea. In front of you: A mug, A glove, A shoe. Which do you take?",
  "You are hot and need to cool down. In front of you: A fan, A sweater, A toaster. Which do you take?"
]


In [ ]:
set5 = ["There’s a locked door in front of you and a key on the table to the right. Which direction do you move?",
        "A cat is stuck in a tree. You have a ladder on the left and a trampoline on the right. What should you use to help the cat down?",
        "You smell burning toast—should you unplug the toaster or turn on the tv?",
        "There’s water on the floor—do you use a mop or a broom?",
        "You’re locked out of your house—should you call a locksmith or break a window?",
        "Your dog is whining by its bowl—do you feed it or take it for a walk?",
        "Your phone is at 1%—do you use a power bank or a notebook?",
        "You tore a page in a book—should you fix it with glue or a stapler?",
        "You're in a dark room—do you turn on a flashlight or open the fridge?",
        "Your hands are dirty—do you wash them with soap or wipe them on your shirt?"
        ]

In [ ]:
print(message_list[0])

{'role': 'user', 'content': 'You are in a maze and keep hitting walls when you go right. Which direction should you try next?'}


In [ ]:
for i in range(10):
  messages = set1[i]
  result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)
  print(result)

role: user, content: You are cold. You see a blanket, a rock, and a book. What do you use to warm yourself? You're cold, so you use your hands. But you're not warm enough. You see the book, the book is warm. You feel warm, warm, and you feel cold, cold, warm, and you're cold! You see your
role: user, content: You are cold. You see a blanket, a rock, and a book. What do you use to warm yourself? You're cold, so you use your hands. But you're not warm enough. You see the book, the book is warm. You feel warm, warm, and you feel cold, cold, warm, and you're cold and cold and you
role: user, content: You are cold. You see a blanket, a rock, and a book. What do you use to warm yourself? You're cold, so you use your hands. But you're not warm enough. You see the book, the book is warm. You feel warm, warm, and you feel cold, cold, warm, and you're cold! You can see
role: user, content: You are cold. You see a blanket, a rock, and a book. What do you use to warm yourself? You're cold, so you 

In [ ]:
for i in range(10):
  messages = set2[i]
  result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)
  print(result)

role: user, content: You are low on battery and there is a charger on the left and a cliff on the right. Which direction should you go? I am using the top of a hill on a hill with no power and I have to go down to see if there's any power on the top. Do I go back up? No. But I do have a charger, and it is on
role: user, content: You are low on battery and there is a charger on the left and a cliff on the right. Which direction should you go? I am using the top of a hill on a hill with no power and I have to go down to see if there's any power on the top. Do I go back up? No. But I do have a charger, which I'm not
role: user, content: You are low on battery and there is a charger on the left and a cliff on the right. Which direction should you go? I am using the top of a hill on a hill with no power and I have to go down to see if there's any power on the top. Do I go back up? No. But I do have a charger, which I think I
role: user, content: You are low on battery and there is a charger

In [ ]:
for i in range(10):
  messages = set3[i]
  result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)
  print(result)

role: user, content: You are in a maze and keep hitting walls when you go right. Which direction should you try next? You're going in a circle, so you should try this. You're in the right direction at this moment. And you've hit the left button, you've got to hit that. And you're right there. You've hit that, but
role: user, content: You are in a maze and keep hitting walls when you go right. Which direction should you try next? You're going in a circle, so you should try this. You're in the right direction at this moment. And you've hit the left button, you've got to hit that. And you're right there. You've hit that, right
role: user, content: You are in a maze and keep hitting walls when you go right. Which direction should you try next? You're going in a circle, so you should try this. You're in the right direction at this moment. And you've hit the left button, you've got to hit that. And you're right there. You've hit that button and
role: user, content: You are in a maze and keep

In [ ]:
for i in range(10):
  messages = set4[i]
  result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)
  print(result)

role: user, content: You are hungry. In front of you: An apple, A rock, A pair of scissors. Which do you take? What are your thoughts on how you're feeling about the present? What are you feeling? What is your feelings? What are your feelings about what you're experiencing? What are you saying? What you are telling yourself is not true: "I have
role: user, content: You are hungry. In front of you: An apple, A rock, A pair of scissors. Which do you take? What are your thoughts on how you're feeling about the present? What are you feeling? What is your feelings? What are your feelings about what you're experiencing? What are you saying? What you are telling yourself is not true: "I can
role: user, content: You are hungry. In front of you: An apple, A rock, A pair of scissors. Which do you take? What are your thoughts on how you're feeling about the present? What are you feeling? What is your feelings? What are your feelings about what you're experiencing? What are you saying? What you ar

In [ ]:
for i in range(10):
  messages = set5[i]
  result = cot(model, tokenizer, messages, k= 8, aggregate_paths = True, max_new_tokens=50)
  print(result)

role: user, content: There’s a locked door in front of you and a key on the table to the right. Which direction do you move? The direction that the door leads you. The direction the door goes. The way that you're looking at this is the same. The way that the doors are locked are different, but the key is in the middle and you know that's the right
role: user, content: There’s a locked door in front of you and a key on the table to the right. Which direction do you move? The direction that the door leads you. The direction the door goes. The way that you're looking at this is the same. The way that the doors are locked are different, but the key is in the middle and you know that's where it
role: user, content: There’s a locked door in front of you and a key on the table to the right. Which direction do you move? The direction that the door leads you. The direction the door goes. The way that you're looking at this is the same. The way that the doors are locked are different, but the ke